# Keras implementation of CapsNet
##### `Dynamic Routing Between Capsules` by Sabour, Frosst and Hinton, in NIPS 2017

### PLAN:
* implement reshaping 3D tensor (output of regular Conv2D into 32 6x6x8 capsules (or preferably, 32 36x8 capsules, so in fact make it (32x36)x8 **DONE**
* implement a capsule layer with forward pass = dynamic routing
    - implement squash function **DONE**
    - implement weight sharing between capsules in each of 32 groups **LOOKS LIKE DONE? NO IDEA IF IT WORKS CORRECTLY THOUGH**
    - implement dynamic routing **SEEMS DONE, BUT TWICE THERE SEEMS TO BE UNNECESSARY DIMS EXPANSION AND COPYING OF TENSORS. TRY FIXING THIS**
* implement margin loss for digit existence **DONE**
* implement reconstruction loss with masking for depending on learning phase
* put it all together into a network
* train on MNIST with small data augmentation as described in the paper, use `.fit_generator()`
* reproduce paper results
* visualize some reconstructions
* visualize how reconstructions change with continous change of one of DigitCaps dimensions

#### NOTE:
Having read the paper again and its parameter count. I think my interpretation of weight sharing is incorrect and in fact each and every of 1152 capsules gets their own Wij matrix for every j, so there should be a total of 11520 weight matrcies of shape 8x16. Delete that later, but first do some experiments to see if your idea wouldn't result in similar performance at reduced cost.

Also, since now each capsule gets its own 8x16 matrix, I don't think my reshaping into primary caps needs to be this complicated anymore. In other words, I don't need to retain the order in which the capsules are laid out in 1152 length vector since then each gets is own matrix anyway, since there is no sharing anymore.

### 1. Reshape

## 2. CapsNet Layer

In [759]:
import numpy as np
from keras import backend as K
from keras import initializers, Input
from keras.models import Model, Sequential
from keras.layers import Activation, Conv2D, Lambda, Dense, Flatten, Reshape
from keras.engine.topology import Layer
from keras.utils.generic_utils import get_custom_objects

import tensorflow as tf

In [732]:
from keras.datasets import mnist
from keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1)
x_train = x_train / 255.

x_test = x_test.reshape(-1, 28, 28, 1)
x_test = x_test / 255.

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [763]:
def get_capsule_network(input_shape=(28, 28, 1)):
    
    # encoder network
    input_tensor = Input(shape=input_shape)
    conv1 = Conv2D(kernel_size=(9,9), strides=(1,1), filters=256, activation='relu')(input_tensor)
    primary_caps = PrimaryCaps(output_capsules=1152, capsule_dim=8, filters=256, kernel_size=(9,9), strides=(2,2))(conv1)
    capsule_layer = CapsuleLayer(output_capsules=10, capsule_dim=16)(primary_caps)
    lengths = Lambda(capsule_length, output_shape=(10,))(capsule_layer)
    
 #  add a layer which in training selects the correct capsule and in test select the capsule with largest length
    input_mask = Input(shape=(10,))
    masked_from_labels = ReconstructionMask()([capsule_layer, input_mask])
    masked_by_length = ReconstructionMask()(capsule_layer)
  
    # decoder network
    decoder = Sequential(name='decoder')
    decoder.add(Dense(512, activation='relu', input_shape=(16,)))
    decoder.add(Dense(1024, activation='relu'))
    decoder.add(Dense(784, activation='sigmoid'))
    decoder.add(Reshape(input_shape))
    
    training_model = Model([input_tensor, input_mask], [lengths, decoder(masked_from_labels)])
    inference_model = Model(input_tensor, [lengths, decoder(masked_by_length)])

    return training_model, inference_model

In [748]:
dummy_X = np.random.random((1, 28, 28, 1))
dummy_mask = np.array([0]*9 + [1]).reshape(1, 10)

CapsNet = get_capsule_network()
output = CapsNet.predict(dummy_X)

In [749]:
caps = output[3]

In [750]:
mask = output[2]

In [491]:
CapsNet.compile(loss=margin_loss(), optimizer='sgd', metrics=['categorical_accuracy'])
CapsNet.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
  128/60000 [..............................] - ETA: 1:19:16 - loss: 0.8100 - categorical_accuracy: 0.0703

KeyboardInterrupt: 

In [456]:
output[0].shape

(1, 10)

In [457]:
output[1].shape

(1, 784)

In [705]:
CapsNet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_142 (InputLayer)          (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_129 (Conv2D)             (None, 20, 20, 256)  20992       input_142[0][0]                  
__________________________________________________________________________________________________
conv2d_130 (Conv2D)             (None, 6, 6, 256)    5308672     conv2d_129[0][0]                 
__________________________________________________________________________________________________
reshape_205 (Reshape)           (None, 1152, 8)      0           conv2d_130[0][0]                 
__________________________________________________________________________________________________
reshape_20

In [489]:
def capsule_length(x):
    length = K.sqrt(K.sum(K.square(x), axis=-1))
    return length

def squash(x):
    l2_norm = K.sum(K.square(x), axis=-1, keepdims=True)
    squash = l2_norm / (1 + l2_norm) * (x / (K.sqrt(l2_norm) + K.epsilon()))
    return squash

get_custom_objects().update({'squash': Activation(squash)})

In [488]:
# y pred is one hot encoded vector so of size (batch_size, num_classes)

def margin_loss(lambda_=0.5, m_plus=0.9, m_minus=0.1):
    def margin(y_true, y_pred):
        loss = K.sum(
            y_true * K.square(K.maximum(0., m_plus - y_pred)) + 
            lambda_ * (1 - y_pred) * K.square(K.maximum(0., y_pred - m_minus)),
            axis=1,
        )
        return K.mean(loss)
    return margin

In [764]:
def reconstruction_loss(y_true, y_pred):
    pass

In [486]:
def PrimaryCaps(
    output_capsules, 
    capsule_dim,
    filters,
    kernel_size, 
    strides=(1,1),
    padding='valid',
):
    
    def eval_primary_caps(input_tensor):
        
        conv2d = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)
        x = conv2d(input_tensor)
        reshaped = Reshape((-1, capsule_dim))(x)
        output_tensor = Reshape((output_capsules, capsule_dim))(reshaped)
        return Lambda(Activation('squash'))(output_tensor)

    return eval_primary_caps

In [487]:
class CapsuleLayer(Layer):
    
    def __init__(
        self, 
        output_capsules, 
        capsule_dim, 
        routing_iterations=3, 
        kernel_initializer='glorot_uniform',
        activation='squash',
        **kwargs
    ):
        self.output_capsules = output_capsules
        self.capsule_dim = capsule_dim
        self.routing_iterations = routing_iterations
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.activation=Activation(activation)
        super(CapsuleLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        
        self.kernel = self.add_weight(
            name='kernel',
            shape=(
                input_shape[1], 
                self.output_capsules, 
                input_shape[2], 
                self.capsule_dim,
            ),
            initializer=self.kernel_initializer,
            trainable=True
        )
        
        super(CapsuleLayer, self).build(input_shape)
        
    def call(self, x):
        x = K.expand_dims(x, axis=2)
        x = K.repeat_elements(x, rep=self.output_capsules, axis=2)
        U = K.map_fn(lambda x: K.batch_dot(x, self.kernel, axes=[2,2]), x)
        
        # initialize matrix of b_ij's
        input_shape = K.shape(x)
        B = K.zeros(shape=(input_shape[0], input_shape[1], self.output_capsules))
        for i in range(self.routing_iterations):
            V, B_updated = self._routing_single_iter(B, U, i, input_shape)
            B = B_updated
            
        return V
            
    def _routing_single_iter(self, B, U, i, input_shape):
        C = K.softmax(B, axis=-1)
        C = K.expand_dims(C, axis=-1)
        C = K.repeat_elements(C, rep=self.capsule_dim, axis=-1)
        S = K.sum(C * U, axis=1)
        V = self.activation(S)
        if i != self.routing_iterations:
            V_expanded = K.expand_dims(V, axis=1)
            V_expanded = K.tile(V_expanded, [1, input_shape[1], 1, 1])
            B = B + K.sum(U*V_expanded, axis=-1)
        return V, B
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_capsules, self.capsule_dim)

In [735]:
class ReconstructionMask(Layer):
    def call(self, input_):
        if type(input_) == list and len(input_) == 2:
            x = input_[0]
            mask = input_[1]
        else:
            x = input_
            len_x = K.sqrt(K.sum(K.square(input_), -1))
            mask = K.one_hot(indices=K.argmax(len_x), num_classes=K.shape(x)[1])
        
        return tf.boolean_mask(x, mask, axis=0)
    
    def compute_output_shape(self, input_):
        if type(input_) == list and len(input_) == 2:
            input_shape = input_[0]
            return (input_shape[0], input_shape[2])
        else:
            return (input_[0], input_[2])

In [673]:
mask = np.array(
[
    [1,0,0,0,0,0,0,0,0,0], 
    [0,0,0,0,0,0,0,0,0,1]
]
)

In [723]:
x = K.random_normal((2, 10, 16))
len_x = K.sqrt(K.sum(K.square(x), -1))
mask = K.one_hot(indices=K.argmax(len_x), num_classes=K.shape(x)[1])

In [724]:
mask

<tf.Tensor 'one_hot:0' shape=(2, ?) dtype=float32>

In [717]:
K.one_hot?

Signature: K.one_hot(indices, num_classes)
Docstring:
Computes the one-hot representation of an integer tensor.

# Arguments
    indices: nD integer tensor of shape
        `(batch_size, dim1, dim2, ... dim(n-1))`
    num_classes: Integer, number of classes to consider.

# Returns
    (n + 1)D one hot representation of the input
    with shape `(batch_size, dim1, dim2, ... dim(n-1), num_classes)`
File:      ~/anaconda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py
Type:      function


In [677]:
import tensorflow as tf

In [680]:
tf.boolean_mask(x, mask, axis=0)

<tf.Tensor 'boolean_mask_1/GatherV2:0' shape=(?, 16) dtype=float32>

In [ ]:
x = K.random_normal

In [301]:
B = K.zeros(shape=(12, 1152, 10))
C =  K.map_fn(lambda x: K.softmax(x, axis=-1), B)

x_batch = K.ones(shape=(12,1152, 8))
x_batch = K.expand_dims(x_batch, axis=2)
x_batch = K.repeat_elements(x_batch, rep=10, axis=2)
y_batch = K.ones(shape=(1152, 10, 8, 16))
xy_batch_dot = K.map_fn(lambda x: K.batch_dot(x, y_batch, axes=[2,2]), x_batch)

C= K.expand_dims(C, axis=-1)
C = K.repeat_elements(C, rep=16, axis=-1)

print(K.int_shape(C))
print(K.int_shape(xy_batch_dot))

S = K.sum(C * xy_batch_dot, axis=1)
S = K.expand_dims(S, axis=1)
S = K.tile(S, [1,1152,1,1])
# S = K.repeat_elements(S, rep=1152, axis=1)

print(K.int_shape(S))
new_B = K.sum(xy_batch_dot*S, axis=-1 )
print(K.int_shape(new_B))

(12, 1152, 10, 16)
(12, 1152, 10, 16)
(12, 1152, 10, 16)
(12, 1152, 10)


In [353]:
dummy_X = np.random.random((32,6,6,256))

input_tensor = Input(shape=input_shape)
reshaped = Reshape((36, 32, 8))(input_tensor)
permuted = Permute((2,1,3))(reshaped)
primary_caps = Reshape(output_shape)(permuted)
digit_caps = CapsuleLayer(10, 16, num_groups=32)(primary_caps)

model = Model(input_tensor, digit_caps)
output = model.predict(dummy_X)

In [354]:
output.shape

(32, 10, 16)

In [327]:
c = CapsuleLayer(10, 16)
c.build((1,1152,8))

In [328]:
c.get_weights()[0].shape

(1152, 10, 8, 16)

In [330]:
c2 = CapsuleLayer(10, 16, num_groups=32)

In [331]:
c2.build((1,1152,8))

In [337]:
W = c2.get_weights()

In [338]:
len(W)

32